# Adesoji Adeshina #
# BIOE 190 Programming Project#
## Comparing topology of phylogenetic trees generated using different techniques ##

In [4]:
### Preamble: imports and stuff like that ###

import scipy
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
from Bio import AlignIO
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from ete3 import Tree
from __future__ import division

aas = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
blosum.update(((b,a),val) for (a,b),val in blosum.items())

BLOSUM62 = np.zeros((20, 20))
for i in xrange(20):
    for j in xrange(20):
        BLOSUM62[i, j] = blosum[(aas[i], aas[j])]
gap_penalties = np.array([-4]*20)

## Part 1 - Multiple Sequence Alignment ## 

In [5]:
#MSA for TLR
tlr_handle = open("tlrs.fasta.txt", "rU")
tlrs = AlignIO.read(tlr_handle, "fasta")

In [6]:
#MSA for KCNA
kcn_handle = open("kcns.fasta.txt", "rU")
kcns = AlignIO.read(kcn_handle, "fasta")

In [7]:
#accessing human readable gene names 
acc_id = {}
tlr_acc = [tlr.name for tlr in tlrs]
kcn_acc = [kcn.name for kcn in kcns]
for line in open("tlrs_with_id.fasta.txt", "rU"):
    if ">" in line:
        arr = line.split("|")
        acc_id[arr[1]] = arr[2].split(" ")[0]
for line in open("kcns_with_id.fasta.txt", "rU"):
    if ">" in line:
        arr = line.split("|")
        acc_id[arr[1]] = arr[2].split(" ")[0]

## Part 2 - Profile Construction ##

In [8]:
def construct_profile(msa, **kwargs):
    if "profile_type" in kwargs:
        if kwargs["profile_type"] == "pseudocount":
            n = kwargs["n"] if "n" in kwargs else 0.1
            return [add_n_profile(msa[i, :], n) for i in xrange(len(msa))]
        if kwargs["profile_type"] == "blosum":
            return [blosum62_profile(msa[i, :]) for i in xrange(len(msa))]
        
def add_n_profile(aligned_seq, n):
    profile = np.ones((20, len(aligned_seq)))
    for i, residue in enumerate(aligned_seq):
        if residue in aas:
            idx = aas.index(residue)
            profile[idx, i] += n
    return Profile(profile/21, "pseudocount")

def blosum62_profile(aligned_seq):
    profile = np.zeros((20, len(aligned_seq)))
    for i, residue in enumerate(aligned_seq):
        if residue in aas:
            idx = aas.index(residue)
            profile[:,i] = BLOSUM62[idx,:]
        else:
            profile[:,i] = gap_penalties
    return Profile(profile, "blosum")

class Profile():
    def __init__(self, data, kind):
        self.data = data
        self.kind = kind
        
    def __repr___(self):
        return self.kind

## Part 3 - Profile Scoring ##

In [9]:
#define all the methods used in profile - profile scoring

def score(profile1, profile2, **kwargs):
    if "scoring_fn" in kwargs:
        if kwargs["scoring_fn"] == "kullback_leibler":
            return kullback_leibler_score(profile1.data, profile2.data)
        if kwargs["scoring_fn"] == "jensen_shannon":
            return jensen_shannon_score(profile1.data, profile2.data)
        return sum(position_scores)
    return

#symmetrized version is D_kl = D(P||Q) + D(Q||P)
def kullback_leibler_score(p, q):
    dpq = np.sum(p*np.log(p/q), axis=0)
    dqp = np.sum(q*np.log(q/p), axis=0)
    return sum(dpq + dqp)

#D_js = 1/2 (D(P||M) + D(Q||M))
#M = 1/2 (P + Q)
def jensen_shannon_score(p, q):
    m = 0.5*(p + q)
    dpm = np.sum(p*np.log(p/m), axis=0)
    dqm = np.sum(q*np.log(q/m), axis=0)
    return sum(0.5*(dpm + dqm))

## Part 3 - Tree Construction##

In [10]:
def merge(profile1, profile2):
    merged_profile = 0.5*(profile1.data + profile2.data)
    return Profile(merged_profile, profile1.kind)

def linkage(i, j, profiles, scoring, children, **kwargs):
    if kwargs["link"] == "centroid":
        return score(profiles[i], profiles[j], scoring_fn=scoring)
    left = children[i]
    right = children[j]
    if kwargs["link"] == "average":
        return 1/(len(left)*len(right)) * sum([sum([score(profiles[x], profiles[y],  scoring_fn=scoring)
                                                  for x in left]) for y in right])
    if kwargs["link"] == "single":
        return min([min([score(profiles[x], profiles[y], scoring_fn=scoring) for x in left]) for y in right])
    if kwargs["link"] == "complete":
        return max([max([score(profiles[x], profiles[y], scoring_fn=scoring) for x in left]) for y in right])
        
        

def agglomeration(profiles, accs, scoring, linkage):
    clusters = len(profiles)
    new_profile_id = len(profiles)-1
    profile_ids = range(len(profiles))
    merged = []
    merged_dict= {}
    reverse_merge = {}
    children = {}
    score_dict = {}
    for idx in xrange(len(profiles)):
        children[idx] = [idx]
    
    while(clusters > 1):
        min_score, closest1, closest2, idx = find_min_score(profiles, profile_ids, scoring, children, linkage)
        merged.append(idx)
        clusters -= 1
        new_profile_id += 1
        merged_dict[idx] = new_profile_id
        reverse_merge[new_profile_id] = idx
        score_dict[new_profile_id] = min_score
        children[new_profile_id] = children[idx[0]] + children[idx[1]]
        profile_ids.remove(idx[0])
        profile_ids.remove(idx[1])
        profile_ids.append(new_profile_id)
        profiles.append(merge(closest1, closest2))
    print merged
    return newick_tree(merged, merged_dict, reverse_merge, accs, score_dict)

def find_min_score(profiles, ids, scoring, children, link):
    min_score = np.inf
    indices = (-1, -1)
    for i in xrange(len(ids)-1):
        for j in xrange(i+1, len(ids)):
            curr_score = linkage(ids[i],ids[j], profiles, scoring, children, link=link)
            if curr_score < min_score:
                min_score = curr_score
                indices = (ids[i], ids[j])
    return (min_score, profiles[indices[0]], profiles[indices[1]], indices)
    

#iterate through the list of tuples and generate the tree in newick format
def newick_tree(merged, merged_dict, reverse_merge, acc, score_dict, **kwargs):
    def get_pairing(label, merged):
        for merger in merged:
            if label in merger:
                return merger
        return (0, 0)
    def edge(a, b, score):
        return "(" + a + "," + b + "):" + str(round(score, 4))
    
    def merge(a, b, thresh, acc, label):
        if a < thresh:
            if b < thresh:
                return edge(acc_id[acc[a]], acc_id[acc[b]], score_dict[label])
            newa, newb = reverse_merge[b]
            return edge(acc_id[acc[a]], merge(newa, newb, thresh, acc, b), score_dict[label])
        if b < thresh:
            newa, newb = reverse_merge[a]
            return edge(merge(newa, newb, thresh, acc, a), acc_id[acc[b]], score_dict[label])
        a1, b1 = reverse_merge[a]
        a2, b2 = reverse_merge[b]
        return edge(merge(a1, b1, thresh, acc, a), merge(a2, b2, thresh, acc, b), score_dict[label])
        
    merger = merged[0]
    left = acc_id[acc[merger[0]]]
    right_val = merger[1]
    while merger in merged_dict:
        label = merged_dict[merger]
        if right_val < len(acc):
            right = acc_id[acc[right_val]]
        else:
            (a, b) = reverse_merge[right_val]
            right = merge(a, b, len(acc), acc, right_val)

        merger = get_pairing(label, merged)
        left = edge(left, right, score_dict[label])
        right_val = merger[0] if label == merger[1] else merger[1]
    out_str = left + ";"
    return out_str
            



In [11]:
#profile generation and tree construction
#rerun this cell to regenerate the profile since profiles are modified everytime a tree is generated

tlr_addone_profiles = construct_profile(tlrs, profile_type="pseudocount")
#tlr_blosum_profiles = construct_profile(tlrs, profile_type="blosum")

kcn_addone_profiles = construct_profile(kcns, profile_type="pseudocount")
#kcn_blosum_profiles = construct_profile(kcns, profile_type="blosum")

tlr_newick_tree = agglomeration(tlr_addone_profiles, tlr_acc, "jensen_shannon", "average")
kcn_newick_tree = agglomeration(kcn_addone_profiles, kcn_acc, "jensen_shannon", "average")

[(16, 17), (18, 20), (37, 38), (46, 47), (13, 14), (36, 48), (23, 25), (22, 52), (12, 15), (19, 24), (21, 53), (55, 56), (39, 51), (31, 32), (49, 50), (54, 57), (11, 61), (42, 58), (40, 41), (26, 60), (8, 10), (27, 28), (9, 65), (6, 7), (33, 34), (44, 59), (2, 3), (4, 72), (62, 68), (35, 64), (63, 70), (71, 76), (0, 1), (5, 73), (66, 74), (75, 77), (78, 79), (67, 80), (69, 82), (83, 84), (30, 45), (81, 85), (43, 87), (29, 88), (86, 89)]
[(3, 4), (1, 2), (5, 33), (10, 12), (6, 35), (15, 16), (7, 37), (27, 28), (0, 34), (13, 14), (38, 42), (20, 21), (17, 43), (25, 40), (11, 36), (30, 31), (8, 39), (19, 23), (18, 50), (9, 49), (44, 51), (41, 52), (26, 29), (47, 54), (22, 56), (46, 57), (48, 58), (55, 59), (53, 60), (45, 61), (24, 62), (32, 63)]


In [12]:
print tlr_newick_tree

(((((((((((((TLR2_HUMAN,TLR2_GORGO):0.0001,(TLR2_PANTR,TLR2_PANPA):0.0002):0.0004,(TLR2_MACMU,TLR2_MACFA):0.0006):0.0037,TLR2_CANLF):0.0143,TLR2_HORSE):0.0161,(TLR2_GIRCA,((TLR2_SHEEP,TLR2_CAPIB):0.0017,((TLR2_CAPHI,TLR2_BUBBU):0.0019,(TLR2_BOSTR,(TLR2_BOVIN,(TLR2_BISBI,TLR2_BOSIN):0.001):0.0012):0.0025):0.0029):0.0051):0.0055):0.0206,(TLR2_CRIGR,TLR2_MOUSE):0.0144):0.0272,(TLR22_CHICK,TLR21_CHICK):0.0158):0.0432,((TLR10_HUMAN,TLR10_BOVIN):0.0179,((TLR1_HUMAN,TLR1_MOUSE):0.0239,(TLR6_MOUSE,(TLR6_DASNO,(TLR6_HUMAN,TLR6_BOVIN):0.0192):0.0204):0.0256):0.0335):0.0484):0.0639,((TLR4_CRIGR,(TLR4_MOUSE,TLR4_RAT):0.0136):0.0213,((TLR4_PIG,(TLR4_BOVIN,TLR4_BOSTR):0.0032):0.0191,((TLR4_PAPAN,(TLR4_PONPY,(TLR4_GORGO,(TLR4_PANPA,TLR4_HUMAN):0.0003):0.0009):0.003):0.0065,(TLR4_HORSE,TLR4_FELCA):0.0189):0.0223):0.0237):0.0317):0.0725,TLR3_MOUSE):0.0775,TLR13_MOUSE):0.0797,(TLR8_HUMAN,TLR7_HUMAN):0.0687):0.0854;


In [13]:
print kcn_newick_tree

((((((((((((((((KCNA2_RAT,KCNA2_MOUSE):0.0,KCNA2_HUMAN):0.0003,KCNA2_RABIT):0.0005,KCNA2_CANLF):0.0009,KCNA2_XENLA):0.0051,KCNA2_ONCMY):0.0084,(KCNA1_HUMAN,(KCNA1_MOUSE,KCNA1_RAT):0.0003):0.001):0.0119,(KCNA3_HUMAN,(KCNA3_RAT,KCNA3_MOUSE):0.0004):0.0047):0.0167,KCNA1_ONCMY):0.0195,(KCNA6_HUMAN,(KCNA6_MOUSE,KCNA6_RAT):0.0009):0.0044):0.022,(KCNA7_HUMAN,KCNA7_MOUSE):0.005):0.023,(KCA10_CHICK,KCA10_HUMAN):0.0134):0.025,((KCNA5_MOUSE,KCNA5_RAT):0.0025,(KCNA5_RABIT,(KCNA5_MUSPF,KCNA5_HUMAN):0.0067):0.0081):0.0092):0.0271,(KCNA4_BOVIN,((KCNA4_RAT,KCNA4_MOUSE):0.0007,(KCNA4_HUMAN,KCNA4_MUSPF):0.0012):0.0022):0.0036):0.0299,KCNAS_DROME):0.0339,KCNSK_CAEEL):0.0396;


In [15]:
t_tlr = Tree(tlr_newick_tree)
t_kcn = Tree(kcn_newick_tree)

In [16]:
print t_tlr


                                       /-TLR2_HUMAN
                                    /-|
                                   |   \-TLR2_GORGO
                                 /-|
                                |  |   /-TLR2_PANTR
                                |   \-|
                              /-|      \-TLR2_PANPA
                             |  |
                             |  |   /-TLR2_MACMU
                           /-|   \-|
                          |  |      \-TLR2_MACFA
                        /-|  |
                       |  |   \-TLR2_CANLF
                       |  |
                       |   \-TLR2_HORSE
                     /-|
                    |  |   /-TLR2_GIRCA
                    |  |  |
                    |  |  |      /-TLR2_SHEEP
                    |   \-|   /-|
                    |     |  |   \-TLR2_CAPIB
                    |     |  |
                    |      \-|      /-TLR2_CAPHI
                    |        |   /-|
                    |      

In [17]:
print t_kcn


                                                /-KCNA2_RAT
                                             /-|
                                          /-|   \-KCNA2_MOUSE
                                         |  |
                                       /-|   \-KCNA2_HUMAN
                                      |  |
                                    /-|   \-KCNA2_RABIT
                                   |  |
                                 /-|   \-KCNA2_CANLF
                                |  |
                              /-|   \-KCNA2_XENLA
                             |  |
                           /-|   \-KCNA2_ONCMY
                          |  |
                          |  |   /-KCNA1_HUMAN
                          |   \-|
                        /-|     |   /-KCNA1_MOUSE
                       |  |      \-|
                       |  |         \-KCNA1_RAT
                       |  |
                       |  |   /-KCNA3_HUMAN
                     /-|   \-|
             

Notes: effects of profile estimation techniques vs scoring functions